# Develop Pipeline

This notebook walks through the investigative steps of each decision made in the process of creating the pipeline for merging and transforming data.

In [1]:
import pandas as pd
import numpy as np

from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

## Import and merge data

In [2]:
# import data and clean it up to be merged
arp = pd.read_csv('data/BrandAverageRetailPrice.csv')
details = pd.read_csv('data/BrandDetails.csv')
totsales = pd.read_csv('data/BrandTotalSales.csv')
totunits = pd.read_csv('data/BrandTotalUnits.csv')

In [3]:
arp.rename(columns = {'Brands': 'Brand',
                    'vs. Prior Period': 'ARP_vsPrior'}, inplace = True)
arp.head()

,Brand,Months,ARP,ARP_vsPrior
0,#BlackSeries,08/2020,15.684913,NaN
1,#BlackSeries,09/2020,NaN,-1.000000
2,#BlackSeries,01/2021,13.611428,NaN
3,#BlackSeries,02/2021,11.873182,-0.127705
4,#BlackSeries,03/2021,NaN,-1.000000


In [4]:
totsales.rename(columns = {'Total Sales ($)': 'totSales'}, inplace = True)
totsales.head()

,Months,Brand,totSales
0,09/2018,10x Infused,"1,711.334232"
1,09/2018,1964 Supply Co.,"25,475.21594500000"
2,09/2018,3 Bros Grow,"120,153.644757"
3,09/2018,3 Leaf,"6,063.5297850000000"
4,09/2018,350 Fire,"631,510.0481550000"


In [5]:
totunits.rename(columns = {'Brands': 'Brand',
                            'Total Units': 'totUnits',
                            'vs. Prior Period': 'totUnits_vsPrior'}, inplace = True)
totunits.head()

,Brand,Months,totUnits,totUnits_vsPrior
0,#BlackSeries,08/2020,"1,616.3390040000000",NaN
1,#BlackSeries,09/2020,NaN,-1.000000
2,#BlackSeries,01/2021,715.5328380000000,NaN
3,#BlackSeries,02/2021,766.669135,0.071466
4,#BlackSeries,03/2021,NaN,-1.000000


In [6]:
# merge all month-level data into one dataframe
monthlev = arp.merge(totsales, how = 'outer', on = ['Brand', 'Months']).merge(totunits, how = 'outer', on = ['Brand', 'Months'])
# remove rows with no information for any of the three sale/price measurements
monthlev = monthlev.dropna(subset = ['ARP', 'totSales', 'totUnits'], how = 'all').reset_index(drop = True)
monthlev.head()

,Brand,Months,ARP,ARP_vsPrior,totSales,totUnits,totUnits_vsPrior
0,#BlackSeries,08/2020,15.684913,NaN,"25,352.135918000000","1,616.3390040000000",NaN
1,#BlackSeries,01/2021,13.611428,NaN,"9,739.4234",715.5328380000000,NaN
2,#BlackSeries,02/2021,11.873182,-0.127705,"9,102.802187000000",766.669135,0.071466
3,101 Cannabis Co.,11/2019,34.066667,NaN,"4,465.040321",131.06772,NaN
4,101 Cannabis Co.,01/2020,34.134929,NaN,"11,790.663567",345.4134480000000,NaN


In [7]:
monthlev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25712 entries, 0 to 25711
Data columns (total 7 columns):
Brand               25712 non-null object
Months              25712 non-null object
ARP                 25279 non-null float64
ARP_vsPrior         22570 non-null float64
totSales            25279 non-null object
totUnits            25712 non-null object
totUnits_vsPrior    22961 non-null float64
dtypes: float64(3), object(4)
memory usage: 1.4+ MB


In [8]:
# cast columns to correct type
# clean out commas in totUnits and totSales
monthlev.loc[:,'totUnits'] = [x.replace(',', '') for x in monthlev.totUnits]
monthlev.loc[:, 'totSales'] = [x.replace(',', '') if type(x) == str else np.NaN for x in monthlev.totSales ]
monthlev = monthlev.astype({'Brand': 'str', 'ARP': 'float', 'ARP_vsPrior': 'float', 
                            'totSales': 'float', 'totUnits': 'float', 'totUnits_vsPrior': 'float'})
monthlev.Months = pd.to_datetime(monthlev.Months)
monthlev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25712 entries, 0 to 25711
Data columns (total 7 columns):
Brand               25712 non-null object
Months              25712 non-null datetime64[ns]
ARP                 25279 non-null float64
ARP_vsPrior         22570 non-null float64
totSales            25279 non-null float64
totUnits            25712 non-null float64
totUnits_vsPrior    22961 non-null float64
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.4+ MB


In [9]:
# compute a rolling average of sales for this brand (average of last WINDOWSIZE months' sales)
WINDOWSIZE = 3

monthlev_roll = pd.DataFrame()

timecols = ['totSales', 'totUnits', 'ARP']

brands = monthlev.loc[:,'Brand'].unique()
for b in brands:
    mli = monthlev.loc[monthlev['Brand'] == b, :].sort_values('Months')
    for col in timecols:
        # compute rolling average column
        rollcolname = 'rolling_avg_' + col
        mli.loc[:,rollcolname] = mli.loc[:,col].rolling(WINDOWSIZE, min_periods = 1).mean()
        # shift the column so the rolling average is only from previous WINDOWSIZE months
        newroll = list(mli.loc[:,rollcolname])[0:-1]
        newroll.insert(0, np.NaN)
        mli.loc[:,rollcolname] = newroll
        # compute previous month column
        prevcolname = 'prev_month_' + col
        prevcol = list(mli.loc[:,col][0:-1])
        prevcol.insert(0, np.NaN)
        mli.loc[:,prevcolname] = prevcol
    monthlev_roll = pd.concat([monthlev_roll, mli], axis = 0, sort = False)

In [10]:
# remove initial time point for each brand; that info is now encoded in the second time point, and they won't have a rolling average
monthlev_roll_clean = monthlev_roll.dropna(subset = ['rolling_avg_' + c for c in timecols] + ['prev_month_' + c for c in timecols])
monthlev_roll_clean = monthlev_roll_clean.drop(['ARP_vsPrior', 'totUnits_vsPrior'], axis = 1)
monthlev_roll_clean

,Brand,Months,ARP,totSales,totUnits,rolling_avg_totSales,prev_month_totSales,rolling_avg_totUnits,prev_month_totUnits,rolling_avg_ARP,prev_month_ARP
1,#BlackSeries,2021-01-01,13.611428,9739.423400,715.532838,25352.135918,25352.135918,1616.339004,1616.339004,15.684913,15.684913
2,#BlackSeries,2021-02-01,11.873182,9102.802187,766.669135,17545.779659,9739.423400,1165.935921,715.532838,14.648170,13.611428
4,101 Cannabis Co.,2020-01-01,34.134929,11790.663567,345.413448,4465.040321,4465.040321,131.067720,131.067720,34.066667,34.066667
5,101 Cannabis Co.,2020-02-01,29.091388,20266.761007,696.658431,8127.851944,11790.663567,238.240584,345.413448,34.100798,34.134929
6,101 Cannabis Co.,2020-03-01,32.293498,30465.470533,943.393328,12174.154965,20266.761007,391.046533,696.658431,32.430995,29.091388
...,...,...,...,...,...,...,...,...,...,...,...
25273,Zoma,2020-07-01,7.532381,12530.392238,1663.536810,14882.584923,5743.278567,2125.192697,1120.790601,6.320437,5.124310
25274,Zoma,2020-08-01,9.220357,5571.298054,604.238880,9268.627594,12530.392238,1622.594748,1663.536810,5.743960,7.532381
25276,Zuma Topicals,2019-09-01,37.860964,17579.084649,464.306316,9874.926545,9874.926545,312.515336,312.515336,31.598214,31.598214
25277,Zuma Topicals,2019-10-01,34.546154,12024.061922,348.057905,13727.005597,17579.084649,388.410826,464.306316,34.729589,37.860964


In [11]:
# aggregate details on each product/brand to add to the monthly data
print(details.columns)
details.head()

Index(['State', 'Channel', 'Category L1', 'Category L2', 'Category L3',
       'Category L4', 'Category L5', 'Brand', 'Product Description',
       'Total Sales ($)', 'Total Units', 'ARP', 'Flavor', 'Items Per Pack',
       'Item Weight', 'Total THC', 'Total CBD', 'Contains CBD', 'Pax Filter',
       'Strain', 'Is Flavored', 'Mood Effect', 'Generic Vendor',
       'Generic Items', '$5 Price Increment'],
      dtype='object')


,State,Channel,Category L1,Category L2,Category L3,Category L4,Category L5,Brand,Product Description,Total Sales ($),...,Total THC,Total CBD,Contains CBD,Pax Filter,Strain,Is Flavored,Mood Effect,Generic Vendor,Generic Items,$5 Price Increment
0,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),"1,103.964857",...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
1,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),674.645211,...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
2,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"2,473.699102",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
3,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"14,589.916417",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
4,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Afghan Kush - Wax,145.39627,...,0,0,THC Only,NaN,Afghan Kush,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$35.00 to $39.99


In [12]:
# clean up the column names
details.rename(columns = {'Category L1': 'cat1', 'Category L2': 'cat2', 'Category L3': 'cat3', 'Category L4': 'cat4', 'Category L5': 'cat5',
                'Product Description': 'product', 'Total Sales ($)': 'totSales_2021', 'Total Units': 'totUnits_2021', 'ARP': 'ARP_2021', 
                'Items Per Pack': 'items_per_pack', 'Item Weight': 'item_weight', 'Total THC': 'tot_THC', 'Total CBD': 'tot_CBD', 'Contains CBD': 'cont_CBD', 
                'Pax Filter': 'pax_filter', 'Is Flavored': 'is_flavored', 'Mood Effect': 'mood_effect', 'Generic Vendor': 'generic_vendor', 
                'Generic Items': 'generic_items', '$5 Price Increment': 'price_inc_5dollar'}, inplace = True)
print(details.columns)
details.head()

Index(['State', 'Channel', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'Brand',
       'product', 'totSales_2021', 'totUnits_2021', 'ARP_2021', 'Flavor',
       'items_per_pack', 'item_weight', 'tot_THC', 'tot_CBD', 'cont_CBD',
       'pax_filter', 'Strain', 'is_flavored', 'mood_effect', 'generic_vendor',
       'generic_items', 'price_inc_5dollar'],
      dtype='object')


,State,Channel,cat1,cat2,cat3,cat4,cat5,Brand,product,totSales_2021,...,tot_THC,tot_CBD,cont_CBD,pax_filter,Strain,is_flavored,mood_effect,generic_vendor,generic_items,price_inc_5dollar
0,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),"1,103.964857",...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
1,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),674.645211,...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
2,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"2,473.699102",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
3,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"14,589.916417",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
4,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Afghan Kush - Wax,145.39627,...,0,0,THC Only,NaN,Afghan Kush,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$35.00 to $39.99


In [13]:
# get a sense of categorical variables in the details
catfeat = ['State', 'Channel', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'Flavor',
       'cont_CBD',
       'pax_filter', 'is_flavored', 'mood_effect', 'generic_vendor',
       'generic_items', 'price_inc_5dollar']
[pd.value_counts(details[x]) for x in catfeat]

[California    144977
 Name: State, dtype: int64,
 Licensed    144977
 Name: Channel, dtype: int64,
 Inhaleables        121859
 Ingestibles         15554
 Other Cannabis       3074
 Topicals             2567
 All Accessories      1923
 Name: cat1, dtype: int64,
 Concentrates        83534
 Pre-Rolled          20230
 Flower              17377
 Edibles             11287
 Sublinguals          4267
 Other Cannabis       3074
 Topicals             2567
 Devices              1518
 Shake/Trim/Lite       718
 Accessories           296
 Apparel                95
 Non Infused Food       13
 Grow Supplies           1
 Name: cat2, dtype: int64,
 Vape                     44301
 Dabbable Concentrates    39225
 Pre-Rolled               13523
 Hybrid                   12483
 Infused Pre-Rolled        6707
 Tinctures                 4058
 Candy                     3768
 Plants                    2216
 Infused Foods             2215
 Indica                    2173
 Pills                     1862
 Beverag

In [14]:
np.sort(details['price_inc_5dollar'].unique())

array(['$00.00 to $4.99', '$05.00 to $9.99', '$10.00 to $14.99',
       '$15.00 to $19.99', '$20.00 to $24.99', '$25.00 to $29.99',
       '$30.00 to $34.99', '$35.00 to $39.99', '$40.00 to $44.99',
       '$45.00 to $49.99', '$50.00 to $54.99', '$55.00 to $59.99',
       '$60.00 to $64.99', '$65.00 to $69.99', '$70.00 to $74.99',
       '$75.00 to $79.99', '$80.00 to $84.99', '$85.00 to $89.99',
       '$90.00 to $94.99', '$95.00 to $99.99', 'Over $100', 'Zero Value'],
      dtype=object)

In [15]:
# get a sense of numerical features
numfeat = ['item_weight', 'items_per_pack', 'tot_CBD', 'tot_THC', 'totSales_2021', 'totUnits_2021', 'ARP_2021']
details[numfeat].head()

,item_weight,items_per_pack,tot_CBD,tot_THC,totSales_2021,totUnits_2021,ARP_2021
0,NaN,0,0,0,"1,103.964857",85.86394100000000,12.857142
1,NaN,0,0,0,674.645211,42.931971000000000,15.714285
2,NaN,0,0,0,"2,473.699102",157.417226,15.714285
3,NaN,0,0,0,"14,589.916417","1,195.9888350000000",12.199041
4,NaN,0,0,0,145.39627,4.088759,35.560000


In [16]:
# decide on the features we want to keep and get them into one dataframe
keepfeat = ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'Brand',
       'product', 'Flavor',
       'items_per_pack', 'tot_THC', 'tot_CBD', 'cont_CBD',
       'pax_filter', 'Strain', 'is_flavored', 'mood_effect', 'generic_vendor',
       'generic_items']
details_clean = details[keepfeat]
details_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144977 entries, 0 to 144976
Data columns (total 18 columns):
cat1              144977 non-null object
cat2              144977 non-null object
cat3              144245 non-null object
cat4              102618 non-null object
cat5              50135 non-null object
Brand             144977 non-null object
product           144977 non-null object
Flavor            7807 non-null object
items_per_pack    144977 non-null int64
tot_THC           144977 non-null object
tot_CBD           144977 non-null object
cont_CBD          144977 non-null object
pax_filter        44301 non-null object
Strain            115639 non-null object
is_flavored       11287 non-null object
mood_effect       144977 non-null object
generic_vendor    144977 non-null object
generic_items     144977 non-null object
dtypes: int64(1), object(17)
memory usage: 19.9+ MB


In [17]:
details_clean.head()

,cat1,cat2,cat3,cat4,cat5,Brand,product,Flavor,items_per_pack,tot_THC,tot_CBD,cont_CBD,pax_filter,Strain,is_flavored,mood_effect,generic_vendor,generic_items
0,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),NaN,0,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items
1,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),NaN,0,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items
2,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),NaN,0,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items
3,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),NaN,0,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items
4,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Afghan Kush - Wax,NaN,0,0,0,THC Only,NaN,Afghan Kush,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items


In [18]:
# convert binary columns to numeric
binarycols = ['cont_CBD', 'is_flavored', 'mood_effect', 'generic_vendor', 'generic_items', 'pax_filter']

details_clean.loc[:,'cont_CBD'] = details_clean.loc[:,'cont_CBD'].map({'Contains CBD': 1, 'THC Only': 0})
details_clean.loc[:,'is_flavored'] = details_clean.loc[:,'is_flavored'].map({'Flavored': 1, 'Not Flavored': 0})
details_clean.loc[:,'mood_effect'] = details_clean.loc[:,'mood_effect'].map({'Mood Specific': 1, 'Not Mood Specific': 0})
details_clean.loc[:,'generic_vendor'] = details_clean.loc[:,'generic_vendor'].map({'Generic Vendors': 1, 'Non-Generic Vendors': 0})
details_clean.loc[:,'generic_items'] = details_clean.loc[:,'generic_items'].map({'Generic Items': 1, 'Non-Generic Items': 0})
details_clean.loc[:,'pax_filter'] = details_clean.loc[:,'pax_filter'].map({'Pax': 1, 'Not Pax': 0})

details_clean.loc[:,binarycols] = details_clean.loc[:,binarycols].apply(pd.to_numeric)

details_clean.head()

/Users/niko/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,cat1,cat2,cat3,cat4,cat5,Brand,product,Flavor,items_per_pack,tot_THC,tot_CBD,cont_CBD,pax_filter,Strain,is_flavored,mood_effect,generic_vendor,generic_items
0,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),NaN,0,0,0,0,NaN,Vanilla Frosting,NaN,0,0,0
1,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),NaN,0,0,0,0,NaN,Vanilla Frosting,NaN,0,0,0
2,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),NaN,0,0,0,0,NaN,Blueberry Slushy,NaN,0,0,0
3,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),NaN,0,0,0,0,NaN,Blueberry Slushy,NaN,0,0,0
4,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Afghan Kush - Wax,NaN,0,0,0,0,NaN,Afghan Kush,NaN,0,0,0


In [19]:
details_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144977 entries, 0 to 144976
Data columns (total 18 columns):
cat1              144977 non-null object
cat2              144977 non-null object
cat3              144245 non-null object
cat4              102618 non-null object
cat5              50135 non-null object
Brand             144977 non-null object
product           144977 non-null object
Flavor            7807 non-null object
items_per_pack    144977 non-null int64
tot_THC           144977 non-null object
tot_CBD           144977 non-null object
cont_CBD          144977 non-null int64
pax_filter        44301 non-null float64
Strain            115639 non-null object
is_flavored       11287 non-null float64
mood_effect       144977 non-null int64
generic_vendor    144977 non-null int64
generic_items     144977 non-null int64
dtypes: float64(2), int64(5), object(11)
memory usage: 19.9+ MB


In [20]:
# convert remaining numeric columns to numeric
makenumeric = ['tot_THC', 'tot_CBD']
details_clean.loc[:,'tot_THC'] = [x.replace(',', '') for x in details_clean.loc[:,'tot_THC']]
details_clean.loc[:,'tot_CBD'] = [x.replace(',', '') for x in details_clean.loc[:,'tot_CBD']]

details_clean.loc[:,makenumeric] = details_clean.loc[:,makenumeric].apply(pd.to_numeric)
details_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144977 entries, 0 to 144976
Data columns (total 18 columns):
cat1              144977 non-null object
cat2              144977 non-null object
cat3              144245 non-null object
cat4              102618 non-null object
cat5              50135 non-null object
Brand             144977 non-null object
product           144977 non-null object
Flavor            7807 non-null object
items_per_pack    144977 non-null int64
tot_THC           144977 non-null float64
tot_CBD           144977 non-null float64
cont_CBD          144977 non-null int64
pax_filter        44301 non-null float64
Strain            115639 non-null object
is_flavored       11287 non-null float64
mood_effect       144977 non-null int64
generic_vendor    144977 non-null int64
generic_items     144977 non-null int64
dtypes: float64(4), int64(5), object(9)
memory usage: 19.9+ MB


In [21]:
# group on brand and aggregate details how I want to

# define columns I want to do top X proportion aggregation on 
topXcols = ['cat1', 'cat2', 'cat3']

# group details on brand name
details_grouponbrand = details_clean.groupby(['Brand'])

# get the X most frequent values in each column we're doing topX on
NTOP = 5
def topX(column, x):
    sortedcounts = sorted(Counter(column.dropna()).items(), 
                            key = lambda item: item[1], reverse = True)
    return([pair[0] for pair in sortedcounts[0:x]])

topX_percol_dict = {}
for col in topXcols:
    topX_percol_dict[col] = topX(details_clean[col], NTOP)
print(topX_percol_dict)

# get the proportions of the top X values in brand's subset of a feature
def topXproportion(column, topXvals):
    thislen = len(column.dropna()) # get proportion of non-null values that are val
    if(thislen):
        props = [sum(column == val) / thislen for val in topXvals]
        props.append(1 - sum(props)) # add an other column
    else: 
        # return NA if column has none of the most common values
        props = [np.NaN] * (len(topXvals) + 1)
    return(props)

# drop some columns here that I already know have too much missing data
details_agg = details_grouponbrand.agg({'cat1': lambda c: topXproportion(c, topX_percol_dict['cat1']),
                                        'cat2': lambda c: topXproportion(c, topX_percol_dict['cat2']),
                                        'cat3': lambda c: topXproportion(c, topX_percol_dict['cat3']),
                                        'items_per_pack': 'mean',
                                        'tot_THC': 'mean',
                                        'tot_CBD': 'mean',
                                        'cont_CBD': 'mean',	
                                        'pax_filter': 'mean', 
                                        'mood_effect': 'mean', 
                                        'generic_vendor': 'mean',
                                        'generic_items': 'mean'})

details_agg.head()

{'cat1': ['Inhaleables', 'Ingestibles', 'Other Cannabis', 'Topicals', 'All Accessories'], 'cat2': ['Concentrates', 'Pre-Rolled', 'Flower', 'Edibles', 'Sublinguals'], 'cat3': ['Vape', 'Dabbable Concentrates', 'Pre-Rolled', 'Hybrid', 'Infused Pre-Rolled']}


,cat1,cat2,cat3,items_per_pack,tot_THC,tot_CBD,cont_CBD,pax_filter,mood_effect,generic_vendor,generic_items
Brand,,,,,,,,,,,
#BlackSeries,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.5, 0.0, 0.5]",0.000000,0.000000,0.000000,0.000000,NaN,0.0,0,0.000000
101 Cannabis Co.,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.935064935064935, 0.06493506493506493, 0.0, ...","[0.0, 0.935064935064935, 0.0, 0.0, 0.064935064...",0.064935,0.000000,0.000000,0.000000,NaN,0.0,0,0.064935
11:11,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.9132947976878613, 0.08670520231213873, 0.0,...","[0.06936416184971098, 0.8439306358381503, 0.01...",0.086705,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000
19Forty LA,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",1.000000,66.666667,41.666667,0.666667,NaN,0.0,0,0.000000
1Lyfe,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.21568627450980393, 0.7843137254901961,...","[0.0, 0.0, 0.21568627450980393, 0.656862745098...",0.215686,0.000000,0.000000,0.000000,NaN,0.0,0,0.000000


In [22]:
# convert columns of lists into separate columns
notTopXcols = list(set(details_agg.columns) - set(topXcols))
dta = details_agg.loc[:,notTopXcols]
for col in topXcols:
    thisdf = pd.DataFrame(details_agg[col].tolist(), columns = topX_percol_dict[col] + ['other_' + col])
    dta = pd.concat([dta.reset_index(drop = True), thisdf.reset_index(drop = True)], axis = 1)

dta.loc[:,'Brand'] = details_agg.index

dta

,generic_vendor,cont_CBD,tot_THC,pax_filter,tot_CBD,items_per_pack,generic_items,mood_effect,Inhaleables,Ingestibles,...,Edibles,Sublinguals,other_cat2,Vape,Dabbable Concentrates,Pre-Rolled,Hybrid,Infused Pre-Rolled,other_cat3,Brand
0,0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.500000,#BlackSeries
1,0,0.000000,0.000000,NaN,0.000000,0.064935,0.064935,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,0.935065,0.000000,0.000000,0.064935,0.000000,101 Cannabis Co.
2,0,0.000000,0.000000,0.0,0.000000,0.086705,0.000000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.069364,0.843931,0.011561,0.000000,0.075145,0.000000,11:11
3,0,0.666667,66.666667,NaN,41.666667,1.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,19Forty LA
4,0,0.000000,0.000000,NaN,0.000000,0.215686,0.000000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.215686,0.656863,0.000000,0.127451,1Lyfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Zenleaf
1119,0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Zig Zag
1120,0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,Zips Weed Co.
1121,0,0.000000,0.000000,NaN,0.000000,2.333333,0.000000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,0.533333,0.466667,0.000000,0.000000,0.000000,Zkittlez


In [23]:
# add some augmented features
print(dta.columns)

# cross mean total THC and mean mood effect
# this could highlight brands that sell many highly potent products
dta.loc[:,'THC_cross_mood'] = dta.loc[:,'tot_THC'] * dta.loc[:,'mood_effect']

# cross mean generic vendor and proportion of flower products
# this could highlight brands that get their product into a lot of stores 
# and focus on a method of smoking weed that people have been doing for a long time
dta.loc[:,'generic_vendor_cross_flower'] = dta.loc[:,'generic_vendor'] * dta.loc[:,'Flower']

dta

Index(['generic_vendor', 'cont_CBD', 'tot_THC', 'pax_filter', 'tot_CBD',
       'items_per_pack', 'generic_items', 'mood_effect', 'Inhaleables',
       'Ingestibles', 'Other Cannabis', 'Topicals', 'All Accessories',
       'other_cat1', 'Concentrates', 'Pre-Rolled', 'Flower', 'Edibles',
       'Sublinguals', 'other_cat2', 'Vape', 'Dabbable Concentrates',
       'Pre-Rolled', 'Hybrid', 'Infused Pre-Rolled', 'other_cat3', 'Brand'],
      dtype='object')


,generic_vendor,cont_CBD,tot_THC,pax_filter,tot_CBD,items_per_pack,generic_items,mood_effect,Inhaleables,Ingestibles,...,other_cat2,Vape,Dabbable Concentrates,Pre-Rolled,Hybrid,Infused Pre-Rolled,other_cat3,Brand,THC_cross_mood,generic_vendor_cross_flower
0,0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.500000,#BlackSeries,0.0,0.0
1,0,0.000000,0.000000,NaN,0.000000,0.064935,0.064935,0.0,1.0,0.0,...,0.0,0.000000,0.935065,0.000000,0.000000,0.064935,0.000000,101 Cannabis Co.,0.0,0.0
2,0,0.000000,0.000000,0.0,0.000000,0.086705,0.000000,0.0,1.0,0.0,...,0.0,0.069364,0.843931,0.011561,0.000000,0.075145,0.000000,11:11,0.0,0.0
3,0,0.666667,66.666667,NaN,41.666667,1.000000,0.000000,0.0,0.0,0.0,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,19Forty LA,0.0,0.0
4,0,0.000000,0.000000,NaN,0.000000,0.215686,0.000000,0.0,1.0,0.0,...,0.0,0.000000,0.000000,0.215686,0.656863,0.000000,0.127451,1Lyfe,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Zenleaf,0.0,0.0
1119,0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,Zig Zag,0.0,0.0
1120,0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.0,1.0,0.0,...,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,Zips Weed Co.,0.0,0.0
1121,0,0.000000,0.000000,NaN,0.000000,2.333333,0.000000,0.0,1.0,0.0,...,0.0,0.000000,0.533333,0.466667,0.000000,0.000000,0.000000,Zkittlez,0.0,0.0


In [24]:
# finally, merge this aggregated details information onto the time series information
cookies = monthlev_roll_clean.merge(dta, on = 'Brand', how = 'left')

pd.options.display.max_rows = 100
pd.options.display.min_rows = 100

print(cookies.isna().sum())

pd.reset_option('all')

Brand                              0
Months                             0
ARP                                0
totSales                           0
totUnits                           0
rolling_avg_totSales               0
prev_month_totSales                0
rolling_avg_totUnits               0
prev_month_totUnits                0
rolling_avg_ARP                    0
prev_month_ARP                     0
generic_vendor                  3303
cont_CBD                        3303
tot_THC                         3303
pax_filter                     18271
tot_CBD                         3303
items_per_pack                  3303
generic_items                   3303
mood_effect                     3303
Inhaleables                     3303
Ingestibles                     3303
Other Cannabis                  3303
Topicals                        3303
All Accessories                 3303
other_cat1                      3303
Concentrates                    3303
Pre-Rolled                      3303
F

/Users/niko/opt/anaconda3/lib/python3.7/site-packages/pandas/_config/config.py:622: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [25]:
# remove column with majority of the na's, then drop na
cookies = cookies.drop(['pax_filter'], axis = 1)
cookies = cookies.dropna()

# look at our final dataframe
print(cookies.columns)
print(cookies.info())
cookies

Index(['Brand', 'Months', 'ARP', 'totSales', 'totUnits',
       'rolling_avg_totSales', 'prev_month_totSales', 'rolling_avg_totUnits',
       'prev_month_totUnits', 'rolling_avg_ARP', 'prev_month_ARP',
       'generic_vendor', 'cont_CBD', 'tot_THC', 'tot_CBD', 'items_per_pack',
       'generic_items', 'mood_effect', 'Inhaleables', 'Ingestibles',
       'Other Cannabis', 'Topicals', 'All Accessories', 'other_cat1',
       'Concentrates', 'Pre-Rolled', 'Flower', 'Edibles', 'Sublinguals',
       'other_cat2', 'Vape', 'Dabbable Concentrates', 'Pre-Rolled', 'Hybrid',
       'Infused Pre-Rolled', 'other_cat3', 'THC_cross_mood',
       'generic_vendor_cross_flower'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20313 entries, 0 to 23637
Data columns (total 38 columns):
Brand                          20313 non-null object
Months                         20313 non-null datetime64[ns]
ARP                            20313 non-null float64
totSales                       2

,Brand,Months,ARP,totSales,totUnits,rolling_avg_totSales,prev_month_totSales,rolling_avg_totUnits,prev_month_totUnits,rolling_avg_ARP,...,Sublinguals,other_cat2,Vape,Dabbable Concentrates,Pre-Rolled,Hybrid,Infused Pre-Rolled,other_cat3,THC_cross_mood,generic_vendor_cross_flower
0,#BlackSeries,2021-01-01,13.611428,9739.423400,715.532838,25352.135918,25352.135918,1616.339004,1616.339004,15.684913,...,0.0,0.0,0.0,0.000000,0.000000,0.5,0.000000,0.5,0.0,0.0
1,#BlackSeries,2021-02-01,11.873182,9102.802187,766.669135,17545.779659,9739.423400,1165.935921,715.532838,14.648170,...,0.0,0.0,0.0,0.000000,0.000000,0.5,0.000000,0.5,0.0,0.0
2,101 Cannabis Co.,2020-01-01,34.134929,11790.663567,345.413448,4465.040321,4465.040321,131.067720,131.067720,34.066667,...,0.0,0.0,0.0,0.935065,0.000000,0.0,0.064935,0.0,0.0,0.0
3,101 Cannabis Co.,2020-02-01,29.091388,20266.761007,696.658431,8127.851944,11790.663567,238.240584,345.413448,34.100798,...,0.0,0.0,0.0,0.935065,0.000000,0.0,0.064935,0.0,0.0,0.0
4,101 Cannabis Co.,2020-03-01,32.293498,30465.470533,943.393328,12174.154965,20266.761007,391.046533,696.658431,32.430995,...,0.0,0.0,0.0,0.935065,0.000000,0.0,0.064935,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23633,Zkittlez,2021-03-01,39.352005,30241.899130,768.497040,6116.791592,3848.649409,111.146389,61.006881,55.554205,...,0.0,0.0,0.0,0.533333,0.466667,0.0,0.000000,0.0,0.0,0.0
23634,Zkittlez,2021-04-01,39.387355,35209.055568,893.917749,14584.506270,30241.899130,331.358823,768.497040,53.717689,...,0.0,0.0,0.0,0.533333,0.466667,0.0,0.000000,0.0,0.0,0.0
23635,Zkittlez,2021-05-01,40.463240,25006.899159,618.015240,23099.868036,35209.055568,574.473890,893.917749,47.274952,...,0.0,0.0,0.0,0.533333,0.466667,0.0,0.000000,0.0,0.0,0.0
23636,Zkittlez,2021-06-01,38.295832,15835.402614,413.501985,30152.617952,25006.899159,760.143343,618.015240,39.734200,...,0.0,0.0,0.0,0.533333,0.466667,0.0,0.000000,0.0,0.0,0.0
